# Rossmann

Please manually download the competition data from https://www.kaggle.com/c/rossmann-store-sales/data to ./data/raw

In [ ]:
%matplotlib inline
import pandas as pd
import os, sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from guacml import GuacMl

In [ ]:
stores = pd.read_csv('./data/raw/store.csv.zip')
train = pd.read_csv('./data/raw/train.csv.zip')
test = pd.read_csv('./data/raw/test.csv.zip')

In [ ]:
merged = train.merge(stores, how='outer', on=['Store'])
rand_stores = merged.Store.drop_duplicates().sample(10)
sample = merged[merged.Store.isin(rand_stores)]
sample = sample[sample.Sales > 0]

In [ ]:
%time guac = GuacMl(sample, 'Sales', eval_metric='rmspe', exclude_cols=['Customers'])

In [ ]:
guac.info()

In [ ]:
guac.data.df.shape[0]

In [ ]:
guac.make_time_series(date_split_col='Date', series_key_cols='Store', prediction_length=14)
guac.plots.target_plot()

In [ ]:
guac.run(5)

In [ ]:
guac.plots.predictions_vs_actual('xg_boost')

In [ ]:
a = guac.data.df.sort_values(['Store', 'Date'])[:10]
a = a[['Store', 'Date', 'Sales']]
a = a.iloc[[0, 3, 5 ,6]]

In [ ]:
a['index'] = 1
a['level_0'] = 1
a

In [ ]:
a.reset_index()

In [ ]:
print(pd.DateOffset(days=1).kwds)

In [ ]:
a = pd.DataFrame({'a': [1,None,3]})
b = pd.DataFrame({'a': [1,2,3], 'b': [0,0,0]})

In [ ]:
a.append(b)

In [ ]:
a[a.a < 2]